In [18]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import pickle


In [19]:
def scrape_goodreads(category):
    url = f"https://www.goodreads.com/search?utf8=%E2%9C%93&q={category}&search_type=books"
    req = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'})

    content = BeautifulSoup(req.content, 'html.parser')
    books = content.find_all('a', class_='bookTitle')
    authors = content.find_all('a', class_="authorName")

    data = []
    for book, author in zip(books, authors):
        book_name = book.find('span', itemprop='name').text.strip()
        author_name = author.find('span', itemprop='name').text.strip()
        data.append({'Book': book_name, 'Author': author_name})

    return json.dumps(data, indent=4)

In [20]:
# category = input("Enter the searching category - Name of the book / Genre / Author: ")
# json_output = scrape_goodreads(category)
# print(json_output)

In [21]:
import random
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [22]:
data_file = open('capstoneIndentPart1.json').read()
intents = json.loads(data_file)

In [23]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

In [24]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(words), "unique lemmatized words")
print (len(classes), "classes", classes)

28 documents
39 unique lemmatized words
4 classes ['book_search', 'goodbye', 'greeting', 'thanks']


In [25]:
# Create the training data
training_data = []
for doc in documents:
    # Create a bag of words for each document
    bag = [1 if word in doc[0] else 0 for word in words]
    # Add the bag of words and the output row to the training data
    training_data.append([bag, doc[1]])

# Find the maximum length of any list in the training list
max_length = max(len(bag) for bag, _ in training_data)

# Pad all lists in the training list to the maximum length
padded_training = [
    (np.pad(bag, (0, max_length - len(bag)), 'constant'), output_row)
    for bag, output_row in training_data
]
training = np.array(padded_training, dtype=object)

In [26]:
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # Correct bag of words creation
    bag = [1 if w in doc[0] else 0 for w in words]  # This creates a list with 1 or 0 directly
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffle and convert to numpy arrays
random.shuffle(training)
# Convert list of lists to numpy array properly
train_x = np.array([np.array(elem[0]) for elem in training])
train_y = np.array([np.array(elem[1]) for elem in training])

# Now, print shapes to confirm
print("Shape of train_x:", train_x.shape)
print("Shape of train_y:", train_y.shape)

Shape of train_x: (28, 39)
Shape of train_y: (28, 4)


In [27]:
# Build the model
model = Sequential([
    Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(train_y[0]), activation='softmax')
])

sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [28]:
# Fit model
hist = model.fit(train_x, train_y, epochs=500, batch_size=5, verbose=1)

# Save model
model.save('chatbot_model.h5')

print("Model Created")

Epoch 1/500
6/6 [==============================] - 1s 3ms/step - loss: 1.3791 - accuracy: 0.2143
Epoch 2/500
6/6 [==============================] - 0s 4ms/step - loss: 1.3183 - accuracy: 0.4286
Epoch 3/500
6/6 [==============================] - 0s 3ms/step - loss: 1.3115 - accuracy: 0.3214
Epoch 4/500
6/6 [==============================] - 0s 3ms/step - loss: 1.2788 - accuracy: 0.4286
Epoch 5/500
6/6 [==============================] - 0s 3ms/step - loss: 1.2157 - accuracy: 0.3571
Epoch 6/500
6/6 [==============================] - 0s 3ms/step - loss: 1.2269 - accuracy: 0.3571
Epoch 7/500
6/6 [==============================] - 0s 3ms/step - loss: 1.1849 - accuracy: 0.4643
Epoch 8/500
6/6 [==============================] - 0s 3ms/step - loss: 1.1148 - accuracy: 0.5357
Epoch 9/500
6/6 [==============================] - 0s 3ms/step - loss: 1.1541 - accuracy: 0.4286
Epoch 10/500
6/6 [==============================] - 0s 3ms/step - loss: 1.1034 - accuracy: 0.6429
Epoch 11/500
6/6 [===========

In [29]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

    return sentence_words

In [30]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:

                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [31]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.35
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list

In [32]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            if tag == 'book_search':
              category = input("Sure, I'd be happy to recommend a book. What type of book are you in the mood for? ")
              result = scrape_goodreads(category)
            else:
              result = random.choice(i['responses'])
            break

    return result

In [33]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [34]:
chatbot_response('good bye')

1/1 [==============================] - 0s 257ms/step


'Bye! Have a great day!'